In [ ]:
from collections import deque

class Puzzle8:
    def __init__(self, board, parent=None, move=None, depth=0):
        self.board = board  # 3x3 list to represent the puzzle
        self.parent = parent  # For backtracking the solution path
        self.move = move  # Store the move that led to this state
        self.depth = depth  # Depth level in the search tree

    def is_goal(self):
        # Goal state check
        return self.board == [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

    def get_neighbors(self):
        # Generate all possible next states
        neighbors = []
        zero_pos = self.find_zero()
        moves = self.get_possible_moves(zero_pos)

        for move in moves:
            new_board = self.apply_move(zero_pos, move)
            neighbors.append(Puzzle8(new_board, parent=self, move=move, depth=self.depth + 1))
        return neighbors

    def find_zero(self):
        # Find the position of the blank tile (0)
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == 0:
                    return (i, j)

    def get_possible_moves(self, zero_pos):
        # Get possible moves for the zero position
        i, j = zero_pos
        moves = []
        if i > 0: moves.append((-1, 0))  # Move up
        if i < 2: moves.append((1, 0))   # Move down
        if j > 0: moves.append((0, -1))  # Move left
        if j < 2: moves.append((0, 1))   # Move right
        return moves

    def apply_move(self, zero_pos, move):
        # Apply a move and return the new board configuration
        i, j = zero_pos
        di, dj = move
        new_board = [row[:] for row in self.board]  # Deep copy of the board
        new_board[i][j], new_board[i + di][j + dj] = new_board[i + di][j + dj], new_board[i][j]
        return new_board

    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.board])

def bfs_search_with_node_count(initial_state):
    open_list = deque([initial_state])
    closed_list = set()
    nodes_visited = 0  # Counter for nodes visited

    while open_list:
        current_state = open_list.popleft()
        nodes_visited += 1  # Increment node visit count

        if current_state.is_goal():
            return current_state, nodes_visited  # Return goal state and node count

        closed_list.add(str(current_state.board))

        for neighbor in current_state.get_neighbors():
            if str(neighbor.board) not in closed_list:
                open_list.append(neighbor)
                closed_list.add(str(neighbor.board))

    return None, nodes_visited  # Return None if goal not found along with nodes visited

def backtrack_solution(goal_state):
    path = []
    current = goal_state
    while current:
        path.append(current)
        current = current.parent
    path.reverse()
    return path


## Graph Search Agent - Pseudocode:

```
function GraphSearch(initial_state, goal_state):
    open_list = [initial_state]     // list of nodes to be expanded
    closed_list = []               // list of already visited nodes

    while open_list is not empty:
        current_node = open_list.pop(0)  // remove the first node
        if current_node == goal_state:
            return "Goal Found", reconstruct_path(current_node)

        closed_list.append(current_node)

        for each neighbor in get_neighbors(current_node):
            if neighbor not in closed_list and neighbor not in open_list:
                open_list.append(neighbor)
                set_parent(neighbor, current_node)  // backtracking helper

    return "Goal not found"

function reconstruct_path(node):
    path = []
    while node has parent:
        path.insert(0, node)
        node = node.parent
    return path

```


In [ ]:
import time
import tracemalloc
import random

def generate_puzzle_at_depth(goal_state, depth):
    puzzle = Puzzle8(goal_state)
    for _ in range(depth):
        neighbors = puzzle.get_neighbors()
        puzzle = random.choice(neighbors)  # Randomly select a neighbor state
    return puzzle

# goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
# puzzle_at_depth_5 = generate_puzzle_at_depth(goal_state, 5)
# print("Generated Puzzle at Depth 5:")
# print(puzzle_at_depth_5)

def measure_performance_with_nodes(depth):
    goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
    puzzle = generate_puzzle_at_depth(goal_state, depth)

    tracemalloc.start()  # Start memory tracking
    start_time = time.time()  # Start timer

    # Run search algorithm (BFS with node count)
    goal_state, nodes_visited = bfs_search_with_node_count(puzzle)

    end_time = time.time()  # End timer
    memory_used = tracemalloc.get_traced_memory()[1] / 1024  # Memory in KB
    tracemalloc.stop()  # Stop memory tracking

    # Return time, memory, and nodes visited
    return (end_time - start_time) * 1000, memory_used, nodes_visited  # Time in milliseconds

# Example performance run
depths = [1, 2, 3, 4, 5]
for d in depths:
    time_taken, memory_used, nodes_visited = measure_performance_with_nodes(d)
    print(f"Depth {d} -> Time: {time_taken:.2f} ms, Memory: {memory_used:.2f} KB, Nodes Visited: {nodes_visited}")


Generated Puzzle at Depth 5:
2 0 3
1 4 6
7 5 8
Depth 1 -> Time: 0.54 ms, Memory: 2.88 KB, Nodes Visited: 3
Depth 2 -> Time: 1.47 ms, Memory: 12.02 KB, Nodes Visited: 13
Depth 3 -> Time: 1.49 ms, Memory: 12.52 KB, Nodes Visited: 13
Depth 4 -> Time: 0.34 ms, Memory: 3.00 KB, Nodes Visited: 4
Depth 5 -> Time: 1.86 ms, Memory: 13.94 KB, Nodes Visited: 18
